In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time, os
import torch
from torch.utils import data
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.layers.experimental import preprocessing


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

# Load data in

In [ ]:
# Load in data.
train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
# Split
train_y = train_data.label.to_numpy()
train_x = train_data.to_numpy()[0:,1:].reshape(len(train_data),28,28,1)
test_x = test_data.to_numpy().reshape(len(test_data),28,28,1)

# Normalise - to speed up model. Better if values are between [0,1] then [0,255].
train_x = train_x/255
test_x = test_x/255

train_x, dev_x, train_y, dev_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [ ]:
# lets ckeck one image and its actual label
image = train_x[232]
plt.imshow(image.squeeze())
print(train_y[232])

In [ ]:
# # Reshape data
# train_x = train_x.reshape(len(train_x), 28*28)
# dev_x = dev_x.reshape(len(dev_x), 28*28)

# Exploration

# Preprocessing

In [ ]:
train_y

# Model

In [ ]:
model = keras.Sequential([
    
    # First Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same', input_shape=[28, 28, 1]),
    layers.MaxPool2D(2),

    # Second Convolutional Block
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(2),

    # Third Convolutional Block
    layers.Conv2D(filters=256, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(2),
    
    # Fourth Convolutional Block
    layers.Conv2D(filters=512, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(2),
    
    layers.Flatten(),
    layers.Dropout(.2),
    
    layers.Dense(64,activation='relu'),
    layers.Dense(10,activation='sigmoid')
    
])

# model = keras.Sequential(
#     [keras.layers.Dense(100,input_shape=(784,),activation='relu'),
#     keras.layers.Dense(10,activation='sigmoid')])

model.summary()

In [ ]:
#Data Augmentation 
datagen = ImageDataGenerator(
        featurewise_center=False, # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

#datagen.fit(X_train)
train_gen = datagen.flow(train_x,train_y, batch_size=32)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_gen,
    epochs=20,
    verbose=1,
)

In [ ]:
dev_preds = model.predict(dev_x)
dev_preds = np.argmax(dev_preds, axis=1)
dev_preds

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
fig, ax = plt.subplots(figsize=(12, 12))
cm = confusion_matrix(dev_y,dev_preds, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = [0,1,2,3,4,5,6,7,8,9])
disp = disp.plot(ax=ax)
ax.set_title("Confusion Matrix")
plt.show()
%matplotlib inline

In [ ]:
# test_data_new = test_data.to_numpy().reshape(len(test_data),28*28)
test_preds = model.predict(test_x)
test_preds = np.argmax(test_preds, axis=1)
output = pd.DataFrame({'ImageId': range(1,28001), 'Label': test_preds})
output.to_csv('First.csv', index=False)
print("Your submission was successfully saved!")